# Simulating Single Phase Transport 

The point of an OpenPNM simulation is ultimately to compute some transport process.  In this notebook we will cover the following subjects:

| Topics |
| :- |
| Defining conductance |
| Settings boundary conditions |


Start by defining a network.  We'll use the ``Demo`` class which happens to include all the geometrical pore-scale models already. 

In [46]:
import numpy as np
import openpnm as op
pn = op.network.Demo(shape=[5, 5, 1], spacing=5e-5)
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x271c9cacd60>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.coordination_number                                          25 / 25
  2  pore.coords                                                       25 / 25
  3  pore.diameter                                                     25 / 25
  4  pore.max_size                                                     25 / 25
  5  pore.seed                                                         25 / 25
  6  pore.volume                                                       25 / 25
  7  throat.conns                                                      40 / 40
  8  throat.cross_sectional_area                                       40 / 40
  9  

## Define Phase Viscosity

To fully illustrate the process of performing transport calculations, we'll use an empty ``Phase`` object and add all the needed properties manually:

In [47]:
water = op.phase.Phase(network=pn)

Let's assume that we are interested in pressure driven flow. This requires knowing the viscosity of the phase, so let's add a pore-scale model for computing the viscosity of water:

In [48]:
water.add_model(propname='pore.viscosity',
                model=op.models.phase.viscosity.water_correlation)
print(water)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x271c9cbd9f0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.pressure                                                     25 / 25
  2  pore.temperature                                                  25 / 25
  3  pore.viscosity                                                    25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―

And we can check the individual values to verify they make sense:

In [49]:
print(water['pore.viscosity'])

[0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319]


## Basic Conductance Calculation

Determining the conductance of the conduits between each pair of connected pores is the most important part of performing a simulation. The details of conductance models are covered elsewhere. [](XXX)  For this demonstration will assume the very simplest case where all pressure loss occurs in the throats.

Recall the Hagan-Poiseuille equation for fluid flow through a cylindrical tube:

$$ Q = \frac{\pi R^4}{8 \mu L} \Delta P$$

where $R$ and $L$ are the radius and length of the throat, and $\mu$ is the viscosity of the fluid.  Together this prefactor can be referred to as the hydraulic conductance, $g_h$, giving:

$$ Q = g_h \Delta P $$

So the aim is the compute values of $g_h$ for each throat.  We start by doing this manually:

In [50]:
R = pn['throat.diameter']/2
L = pn['throat.length']
mu = water['throat.viscosity']  # See ProTip below
water['throat.hydraulic_conductance'] = np.pi*R**4/(8*mu*L)
print(water['throat.hydraulic_conductance'])

[4.07871382e-15 2.62488557e-15 2.49650596e-15 8.64316529e-16
 1.52364703e-15 6.80412134e-15 7.28698615e-15 2.96891403e-14
 2.06628669e-15 7.43597321e-14 9.14007428e-15 9.36831469e-15
 2.28178331e-14 1.86746628e-14 1.28418216e-14 7.43505687e-15
 9.52724697e-16 8.89047656e-16 1.74003283e-14 1.18269681e-14
 1.21481798e-15 5.26904727e-14 2.19091470e-15 3.60997390e-14
 8.48775132e-16 1.16849657e-15 6.80252617e-14 6.88544696e-15
 9.68594576e-15 2.86012289e-14 1.78574478e-15 3.62521865e-14
 2.05700098e-14 7.68471332e-15 9.05860716e-15 2.33843351e-14
 9.34621654e-16 1.70132562e-14 1.31241221e-14 7.36657361e-15]


----
> 💡 **ProTip!** 
>
> The ``Phase`` class has a special ability to interpolate pore values to throats, and vice-versa. In PNM simulations all the balances are solved for each pore, so the thermodynamic properties like temperature, pressure, etc. are all defined on pores. Consequently, the physical properties are also defined in pores, like viscosity. However, as shown above we often want viscosity values in the throats. OpenPNM provides a shortcut for this, such that if you request a throat property that does not exist, it will attempt to fetch the pores values and do a linear interpolation of values to produce an array of throat values.  There is also a function for this, ``water.interpolate_data('throat.viscosity')``, but the ``water['throat.viscosity']`` shortcut is very convenient.

In [51]:
water.interpolate_data('throat.viscosity')

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

In [52]:
water['throat.viscosity']

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

## Create Algorithm Object

OpenPNM contains a variety of ``Algorithm`` classes in the ``openpnm.algorithms`` module.  Let's initialize a ``StokesFlow`` algorithm, since this simulates pressure driven flow through the network. 

In [53]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x271c9ce1680>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       0 / 25
  2  pore.bc.value                                                      0 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

## Assign Boundary Conditions

As can be seen in the print-out above there are predefined ``'pore.bc'`` arrays, but they contain no valid values, meaning they are all ``nans``. Once we set some boundary conditions, this will change.  Let's apply pressure BCs on one side of the network, and rate BCs on the other:

In [54]:
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x271c9ce1680>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       5 / 25
  2  pore.bc.value                                                      5 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                               25
  2  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

Now we can see there are 5 valid values of each type.  The ``sf`` algorithm will look for ``'throat.hydraulic_conductance'`` on ``water`` by default, so we can just ``run``:

In [55]:
soln = sf.run()

The ``run`` method solves the mass balance around each pore and computes the pressure within each pore that is required to sustain the flow defined by the boundary conditions.  The ``soln`` object that is returned is a dictionary with the key corresponding to the quantity that was solved for.  

In [56]:
print(soln)

{'pore.pressure': SteadyStateSolution([100000.        , 100000.        , 100000.        ,
                     100000.        , 100000.        , 106210.27050397,
                     105849.95521037, 106589.7947894 , 104530.53233244,
                     107340.02831631, 113136.55666764, 110299.0958055 ,
                     111597.08824878, 111255.08964542, 110474.21371183,
                     120951.97877039, 115823.52328237, 118117.34361488,
                     120276.02424136, 119562.37275159, 126553.01853668,
                     159066.70359773, 126889.12984693, 128074.69654368,
                     130017.71158255])}


The reason for the dict format is to provide a consistent API for single components and multiphysics, where multiple different quanties might be solved for.  However, these ``'pore.pressure'`` values are also written to the dictionary of the algorithm object as well:

In [57]:
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x271c9ce1680>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.bc.rate                                                       5 / 25
  2  pore.bc.value                                                      5 / 25
  3  pore.initial_guess                                                25 / 25
  4  pore.pressure                                                     25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                        

Finally we can look at how much pressure was required in the "right" pores to meet the required flow rate:

In [58]:
sf['pore.pressure'][pn.pores('right')]

array([126553.01853668, 159066.70359773, 126889.12984693, 128074.69654368,
       130017.71158255])

So we can see that 13 kPa was required to accomplish the requested flow. 

## Rigorous Conductance Calculation: Manual Method

The above demonstration used a very simplistic conductance calculation.  It was also stated that computing conductance is the most important part of doing a PNM simulation.  To finish this notebook, we'll look more closely at this process.

Let's print the network object again:

In [63]:
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x271c9cacd60>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.coordination_number                                          25 / 25
  2  pore.coords                                                       25 / 25
  3  pore.diameter                                                     25 / 25
  4  pore.max_size                                                     25 / 25
  5  pore.seed                                                         25 / 25
  6  pore.volume                                                       25 / 25
  7  throat.conns                                                      40 / 40
  8  throat.cross_sectional_area                                       40 / 40
  9  

Note the ``'throat.hydraulic_size_factors'`` array.  This is computed by a pore-scale model on the ``Demo`` network.  This computation is more rigorous in the following ways:

1. The conductance of each half pore and that throat is considered.
1. The throat length is computed carefully by accounting for the 'lens' between the intersection of the spherical pore bodies and the cylindrical throat.
1. The net cross-sectional area of the pores are computed by integrating between the pore center and the pore-throat intersection point

The conductance of each element in the conduit is returned as an *Nt-by-3* array, where columns 1 and 3 contain the hydraulic conductance of the half pore on either end of the throat, and the column 1 contains the throat conductance.

In [60]:
pn['throat.hydraulic_size_factors']

array([[8.71374824e-17, 3.64307026e-18, 1.69476709e-16],
       [1.39107321e-16, 2.34452403e-18, 6.35649123e-17],
       [6.35649123e-17, 2.22985652e-18, 1.23583518e-16],
       [7.63303754e-17, 7.71999699e-19, 2.95278044e-17],
       [4.22927679e-17, 1.36090773e-18, 1.12938944e-16],
       [2.21058166e-16, 6.07737959e-18, 1.23253419e-16],
       [1.23253419e-16, 6.50867006e-18, 2.49810123e-16],
       [4.59253634e-16, 2.65180714e-17, 3.27922318e-16],
       [5.09514999e-17, 1.84558857e-18, 1.38817555e-16],
       [6.65404041e-16, 6.64174396e-17, 6.23438780e-16],
       [2.56148448e-16, 8.16383161e-18, 1.50558689e-16],
       [1.50558689e-16, 8.36769388e-18, 2.67735776e-16],
       [3.06086072e-16, 2.03806820e-17, 3.31572166e-16],
       [3.03338786e-16, 1.66800397e-17, 2.65843221e-16],
       [2.30436485e-16, 1.14701988e-17, 2.12521168e-16],
       [1.68159063e-16, 6.64092549e-18, 1.47038361e-16],
       [8.35046328e-17, 8.50965075e-19, 3.11945583e-17],
       [3.11945583e-17, 7.94089

This data is called the ``size factor`` because it is purely the geometrical information required for the computation of the hydraulic conductance.  So the Hagan-Poisseiulle equation for each element is written as:

$$ Q = \frac{F_h}{\mu} \Delta P = g_h \Delta P$$

Note that both the $\frac{\pi R^4}{8}$ term and $L$ are missing, as they now rolled into the $F_h$ value.

The total conductance of the pore-throat-pore conduit can be found as the sum of three resistors in series.  Since we have conductance values, we add the inverses, and invert again.  The full expression for the hydraulic conductance between pores i and j, through throat k, is:


$$ Q = \bigg( \frac{\mu}{F_{h, i}} + \frac{\mu}{F_{h, k}} + \frac{\mu}{F_{h, j}} \bigg) ^ {-1} \Delta P $$

This can be computed by hand:

In [61]:
F_h = water['throat.hydraulic_size_factors']
water['throat.hydraulic_conductance'] = (mu * (1/F_h).sum(axis=1))**(-1)

In [62]:
water['throat.hydraulic_conductance']

array([3.83588577e-15, 2.49102298e-15, 2.37057327e-15, 8.34074035e-16,
       1.45911308e-15, 6.31883354e-15, 6.75432842e-15, 2.60749307e-14,
       1.96879675e-15, 6.16403034e-14, 8.41553796e-15, 8.61984356e-15,
       2.02276499e-14, 1.67076383e-14, 1.16347406e-14, 6.85475709e-15,
       9.18315496e-16, 8.57350807e-16, 1.55882592e-14, 1.07610967e-14,
       1.16217262e-15, 4.47028330e-14, 2.07688885e-15, 3.13756412e-14,
       8.19194147e-16, 1.11524150e-15, 5.67633404e-14, 6.39262689e-15,
       8.90232105e-15, 2.51610116e-14, 1.70509843e-15, 3.14963896e-14,
       1.83743003e-14, 7.07743827e-15, 8.34359646e-15, 2.07263836e-14,
       9.01053673e-16, 1.52365904e-14, 1.18943332e-14, 6.78971503e-15])

In [65]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([130241.87741162, 163598.56582326, 130374.11254203, 131602.86219683,
       133644.23156533])

As can be seen the numbers are about the same as with the simple case, but should be somewhat more correct.  In fact, these above pressures are a bit higher, which is because the total conductance of the conduit is lower due to the inclusion of the pore body lengths into the total length, compared to above where only the throat length was included.

## Rigorous Conductance Calculation: Pore-Scale Model Method

Instead of computing the hydraulic conductance manually as done above, there is a pore-scale model available:

In [66]:
water.add_model(propname='throat.hydraulic_conductance',
                model=op.models.physics.hydraulic_conductance.generic_hydraulic)

In [67]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([130241.87741162, 163598.56582326, 130374.11254203, 131602.86219683,
       133644.23156533])

Which gives exactly the same result, without have to manually deal with the conductances-in-series calculation.